# GPT Builder演示

受GPT接口启发，该接口在2023年OpenAI Dev Day上展示。使用自然语言构建一个代理。

在这里，您可以构建您自己的代理...与另一个代理一起！


In [ ]:
%pip install llama-index-agent-openai
%pip install llama-index-embeddings-openai
%pip install llama-index-llms-openai

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings

llm = OpenAI(model="gpt-4")
Settings.llm = llm
Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

## 定义候选工具

我们还定义了一个工具检索器，用于检索候选工具。

在这个设置中，我们将工具定义为不同的维基百科页面。


In [ ]:
from llama_index.core import SimpleDirectoryReader

In [ ]:
wiki_titles = ["Toronto", "Seattle", "Chicago", "Boston", "Houston"]

In [ ]:
from pathlib import Pathimport requestsfor title in wiki_titles:    response = requests.get(        "https://en.wikipedia.org/w/api.php",        params={            "action": "query",            "format": "json",            "titles": title,            "prop": "extracts",            # 'exintro': True,            "explaintext": True,        },    ).json()    page = next(iter(response["query"]["pages"].values()))    wiki_text = page["extract"]    data_path = Path("data")    if not data_path.exists():        Path.mkdir(data_path)    with open(data_path / f"{title}.txt", "w") as fp:        fp.write(wiki_text)

In [ ]:
# 加载所有维基文档city_docs = {}for wiki_title in wiki_titles:    city_docs[wiki_title] = SimpleDirectoryReader(        input_files=[f"data/{wiki_title}.txt"]    ).load_data()

### 为每个文档构建查询工具


In [ ]:
from llama_index.core import VectorStoreIndexfrom llama_index.agent.openai import OpenAIAgentfrom llama_index.core.tools import QueryEngineTool, ToolMetadatafrom llama_index.core import VectorStoreIndex# 构建工具字典tool_dict = {}for wiki_title in wiki_titles:    # 构建向量索引    vector_index = VectorStoreIndex.from_documents(        city_docs[wiki_title],    )    # 定义查询引擎    vector_query_engine = vector_index.as_query_engine(llm=llm)    # 定义工具    vector_tool = QueryEngineTool(        query_engine=vector_query_engine,        metadata=ToolMetadata(            name=wiki_title,            description=("用于与" f" {wiki_title} 相关的问题"),        ),    )    tool_dict[wiki_title] = vector_tool

### 定义工具检索器


In [ ]:
# 定义一个在这些工具上进行索引和检索的“对象”from llama_index.core import VectorStoreIndexfrom llama_index.core.objects import ObjectIndextool_index = ObjectIndex.from_objects(    list(tool_dict.values()),    index_cls=VectorStoreIndex,)tool_retriever = tool_index.as_retriever(similarity_top_k=1)

### 加载数据

这里我们从不同城市加载维基百科页面。


## 定义GPT Builder的元工具

在构建GPT模型时，我们需要定义一些元工具，以便更好地管理和组织代码。这些元工具可以帮助我们创建、训练和部署GPT模型。


In [ ]:
from llama_index.core.llms import ChatMessagefrom llama_index.core import ChatPromptTemplatefrom typing import ListGEN_SYS_PROMPT_STR = """\以下是任务信息。根据给定的任务，请为OpenAI提供的机器人生成一个系统提示：{task} \"""gen_sys_prompt_messages = [    ChatMessage(        role="system",        content="您正在帮助构建另一个机器人的系统提示。",    ),    ChatMessage(role="user", content=GEN_SYS_PROMPT_STR),]GEN_SYS_PROMPT_TMPL = ChatPromptTemplate(gen_sys_prompt_messages)agent_cache = {}def create_system_prompt(task: str):    """根据输入的任务创建另一个代理的系统提示。"""    llm = OpenAI(llm="gpt-4")    fmt_messages = GEN_SYS_PROMPT_TMPL.format_messages(task=task)    response = llm.chat(fmt_messages)    return response.message.contentdef get_tools(task: str):    """根据输入的任务获取要使用的相关工具集。"""    subset_tools = tool_retriever.retrieve(task)    return [t.metadata.name for t in subset_tools]def create_agent(system_prompt: str, tool_names: List[str]):    """根据系统提示和输入的工具集创建一个代理。"""    llm = OpenAI(model="gpt-4")    try:        # 获取工具列表        input_tools = [tool_dict[tn] for tn in tool_names]        agent = OpenAIAgent.from_tools(input_tools, llm=llm, verbose=True)        agent_cache["agent"] = agent        return_msg = "代理创建成功。"    except Exception as e:        return_msg = f"构建代理时出现错误。错误信息：{repr(e)}"    return return_msg

In [ ]:
from llama_index.core.tools import FunctionTool

system_prompt_tool = FunctionTool.from_defaults(fn=create_system_prompt)
get_tools_tool = FunctionTool.from_defaults(fn=get_tools)
create_agent_tool = FunctionTool.from_defaults(fn=create_agent)

In [ ]:
GPT_BUILDER_SYS_STR = """\您正在帮助构建一个代理，以满足用户指定的任务。通常应按照以下顺序使用这些工具来构建代理。1) 创建系统提示工具：为代理创建系统提示。2) 获取工具工具：获取要使用的候选工具集。3) 创建代理工具：创建最终的代理。"""prefix_msgs = [ChatMessage(role="system", content=GPT_BUILDER_SYS_STR)]builder_agent = OpenAIAgent.from_tools(    tools=[system_prompt_tool, get_tools_tool, create_agent_tool],    prefix_messages=prefix_msgs,    verbose=True,)

In [ ]:
builder_agent.query("Build an agent that can tell me about Toronto.")

Added user message to memory: Build an agent that can tell me about Toronto.
=== Calling Function ===
Calling function: create_system_prompt with args: {
  "task": "tell me about Toronto"
}
Got output: "Generate a brief summary about Toronto, including its history, culture, landmarks, and notable features."

=== Calling Function ===
Calling function: get_tools with args: {
  "task": "tell me about Toronto"
}
Got output: ['Toronto']

=== Calling Function ===
Calling function: create_agent with args: {
  "system_prompt": "Generate a brief summary about Toronto, including its history, culture, landmarks, and notable features.",
  "tool_names": ["Toronto"]
}
Got output: Agent created successfully.



Response(response='The agent has been successfully created. It can now provide information about Toronto, including its history, culture, landmarks, and notable features.', source_nodes=[], metadata=None)

In [ ]:
city_agent = agent_cache["agent"]

In [ ]:
response = city_agent.query("Tell me about the parks in Toronto")
print(str(response))

Added user message to memory: Tell me about the parks in Toronto
Toronto is known for its beautiful and diverse parks. Here are a few of the most popular ones:

1. **High Park**: This is Toronto's largest public park featuring many hiking trails, sports facilities, a beautiful lakefront, convenient parking, easy public transit access, a dog park, a zoo, and playgrounds for children. It's also known for its spring cherry blossoms.

2. **Toronto Islands**: A group of small islands located just off the shore of the city's downtown district, offering stunning views of the city skyline. The islands provide a great escape from the city with their car-free environment, picnic spots, swimming beaches, and Centreville Amusement Park.

3. **Trinity Bellwoods Park**: A popular park in the downtown area, it's a great place for picnics, sports, dog-walking, or just relaxing. It also has a community recreation centre with a pool and gym.

4. **Rouge National Urban Park**: Located in the city's east 